In [48]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [49]:
CURR_DIR=os.getcwd()
print(CURR_DIR)

DATA_DIR=os.path.join(CURR_DIR,'project_files')
print(DATA_DIR)

data_path=os.path.join(DATA_DIR, 'df_final_v1.csv')
print(data_path)

/Users/dare_devil/Desktop/MLDS_2024/Quarter1/Everything_Starts_With_Data/Project/MLDS_400_G11
/Users/dare_devil/Desktop/MLDS_2024/Quarter1/Everything_Starts_With_Data/Project/MLDS_400_G11/project_files
/Users/dare_devil/Desktop/MLDS_2024/Quarter1/Everything_Starts_With_Data/Project/MLDS_400_G11/project_files/df_final_v1.csv


In [50]:
df_orig = pd.read_csv(data_path)
df_orig.head()

,STORE,COUNT_ITEMS,MenRatio,White,Asian,Income,Poverty,Professional,Construction,Unemployment,...,COST,RETAIL,avg_quan,avg_orig,avg_sprice,purchase_count,return_count,purchase_prob,PROF_MARG,DISCOUNTED
0,102,96312,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,8.400000,13.018372,1.0,17.50,17.500000,2,0,1.000000,0.645242,False
1,102,96312,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,16.000000,19.990000,1.0,35.00,6.496000,4,1,0.800000,0.800400,True
2,102,96312,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,9.600000,5.230000,1.0,20.00,9.400000,5,0,1.000000,1.835564,True
3,102,96312,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,5.420000,5.974000,1.0,25.00,7.186667,5,1,0.833333,0.907265,True
4,102,96312,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,13.493841,20.058913,1.0,24.99,7.578571,14,0,1.000000,0.672710,True


In [51]:
df = df_orig.copy()

In [52]:
df.keys()

Index(['STORE', 'COUNT_ITEMS', 'MenRatio', 'White', 'Asian', 'Income',
       'Poverty', 'Professional', 'Construction', 'Unemployment', 'SKU',
       'DEPT', 'PACKSIZE', 'VENDOR', 'BRAND', 'CLASSID', 'COST', 'RETAIL',
       'avg_quan', 'avg_orig', 'avg_sprice', 'purchase_count', 'return_count',
       'purchase_prob', 'PROF_MARG', 'DISCOUNTED'],
      dtype='object')

In [53]:
categorical_variables = [
    "DEPT",
   # "VENDOR",
    "BRAND",
    "CLASSID",
    "DISCOUNTED"
]

print(len(categorical_variables))

4


In [54]:
def filter_categorical_values(df, var, count_threshold=100):
    """
    Filtering Categorical Variables based on Count
    Basically don't want to keep low count for a categorical variable 
    """
    count_df = df[var].value_counts().reset_index()
    
    keep_vals = count_df[count_df['count'] >= count_threshold][var].tolist()
    drop_vals = count_df[count_df['count'] < count_threshold][var].tolist()
    
    df = df[df[var].isin(keep_vals)]
    print(f"Threshold={count_threshold}, Variable={var}, KeepVals={len(keep_vals)}, DropVals={len(drop_vals)}, Shape={df.shape[0]}")
    return df

In [55]:
print(f"InitialShape={df.shape[0]}")
for var in categorical_variables:
    df = filter_categorical_values(df, var)
print(f"FinalShape={df.shape[0]}")

InitialShape=8645544
Threshold=100, Variable=DEPT, KeepVals=59, DropVals=1, Shape=8645533
Threshold=100, Variable=BRAND, KeepVals=631, DropVals=552, Shape=8632703
Threshold=100, Variable=CLASSID, KeepVals=582, DropVals=182, Shape=8626635
Threshold=100, Variable=DISCOUNTED, KeepVals=2, DropVals=0, Shape=8626635
FinalShape=8626635


In [56]:
df_full_purchase = df[df.purchase_prob == 1.000000]
df_bet = df[(df.purchase_prob != 1.000000) & (df.purchase_prob != 0)]
df_zero_purchse = df[df.purchase_prob == 0]

print(len(df_full_purchase), len(df_bet), len(df_zero_purchse))

6218913 2403059 4663


In [57]:
df_full_purchase = df[df.purchase_prob == 1.000000]
df_rest = df[(df.purchase_prob != 1.000000)]


print()

count = 0
sample_count = 6000
total_vars = len(categorical_variables)

def equate_condition(df1, df2, var, apply_condition2 = True, threshold=20):
    
    cond1 = df1[var].unique().shape[0] == df2[var].unique().shape[0]
    
    if apply_condition2:
        cond2 = df2[var].value_counts().reset_index()['count'].min() >= threshold
    else:
        cond2 = True
    
    if cond1 and cond2:
        return 1
    return 0

while True:
    df_full_purchase_sample = df_full_purchase.sample(n=sample_count)
    df_sample = pd.concat([df_rest,df_full_purchase_sample])
    for var in categorical_variables:
        count+= equate_condition(df, df_sample, var)
    if count == total_vars:
        break

In [58]:
df_sample.shape[0], df.shape[0]

(2413722, 8626635)

In [59]:
df_sample.head()

,STORE,COUNT_ITEMS,MenRatio,White,Asian,Income,Poverty,Professional,Construction,Unemployment,...,COST,RETAIL,avg_quan,avg_orig,avg_sprice,purchase_count,return_count,purchase_prob,PROF_MARG,DISCOUNTED
1,102,96312,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,16.00,19.990000,1.0,35.0,6.496000,4,1,0.800000,0.800400,True
3,102,96312,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,5.42,5.974000,1.0,25.0,7.186667,5,1,0.833333,0.907265,True
6,102,96312,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,9.50,21.383839,1.0,28.0,24.253333,5,1,0.833333,0.444261,True
7,102,96312,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,30.00,68.842893,1.0,69.0,46.990002,4,1,0.800000,0.435775,True
8,102,96312,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,6.30,7.965314,1.0,16.0,15.248750,6,2,0.750000,0.790929,True


In [60]:
from sklearn.model_selection import train_test_split

In [61]:
TEST_SPLIT=0.4

data = df_sample.copy()


data['combined'] = data[categorical_variables].astype(str).agg('-'.join, axis=1)



singles = data['combined'].value_counts()[data['combined'].value_counts() == 1].index.tolist()

# Split the data where rare combinations are separated
single_data = data[data['combined'].isin(singles)]
common_data = data[~data['combined'].isin(singles)]

# Stratified split for common_data
train_common, test_common = train_test_split(common_data, stratify=common_data['combined'], test_size=TEST_SPLIT, random_state=42)

# Simple split for single_data (or handle separately)
train_single, test_single = train_test_split(single_data, test_size=TEST_SPLIT, random_state=42)  # Or adjust how you handle this

# Concatenate the results
train = pd.concat([train_common, train_single])
test = pd.concat([test_common, test_single])

# Drop the combined column
train = train.drop(columns=['combined'])
test = test.drop(columns=['combined'])

print(train.shape)
print(test.shape)

(1448232, 26)
(965490, 26)


In [62]:
def equate_condition(df1, df2, var, apply_condition2 = True, threshold=20):
    
    cond1 = df1[var].unique().shape[0] == df2[var].unique().shape[0]
    
    if apply_condition2:
        cond2 = df2[var].value_counts().reset_index()['count'].min() >= threshold
    else:
        cond2 = True
    
    if cond1 and cond2:
        return 1
    return 0

for var in categorical_variables:
    print(var)
    flag = equate_condition(df1=df_sample, df2=train, var=var, apply_condition2 = False)
    assert flag == 1

DEPT
BRAND
CLASSID
DISCOUNTED


In [63]:
train.drop(columns=['purchase_count','return_count'],axis=1, inplace=True)
test.drop(columns=['purchase_count','return_count'],axis=1,inplace=True)

In [64]:
train.head()

,STORE,COUNT_ITEMS,MenRatio,White,Asian,Income,Poverty,Professional,Construction,Unemployment,...,BRAND,CLASSID,COST,RETAIL,avg_quan,avg_orig,avg_sprice,purchase_prob,PROF_MARG,DISCOUNTED
8438940,9409,66634,0.517922,74.657813,1.109375,54353.562500,13.234375,35.031250,14.535937,5.793750,...,VANITY F,272,9.31,20.000000,1.0,20.0,20.000000,0.846154,0.465500,False
3248046,3107,44581,0.509057,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,...,CABERNET,872,8.05,10.893302,1.0,22.0,14.990000,0.750000,0.738986,True
5506591,5404,50013,0.487016,65.168657,0.711940,40271.402985,20.949254,28.095522,11.605970,9.023881,...,POLO FAS,226,37.50,34.533148,1.0,75.0,18.750000,0.750000,1.085913,True
1479941,1402,56215,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,EMMA JAM,006,19.27,16.329684,1.0,49.0,34.028001,0.600000,1.180060,True
3844504,3607,52901,0.509057,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,...,VAN MAR,103,2.10,4.035000,1.0,6.0,5.150000,0.750000,0.520446,True


In [65]:
train['train_type'] = 'train'
test['train_type'] = 'test'

train.head()

,STORE,COUNT_ITEMS,MenRatio,White,Asian,Income,Poverty,Professional,Construction,Unemployment,...,CLASSID,COST,RETAIL,avg_quan,avg_orig,avg_sprice,purchase_prob,PROF_MARG,DISCOUNTED,train_type
8438940,9409,66634,0.517922,74.657813,1.109375,54353.562500,13.234375,35.031250,14.535937,5.793750,...,272,9.31,20.000000,1.0,20.0,20.000000,0.846154,0.465500,False,train
3248046,3107,44581,0.509057,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,...,872,8.05,10.893302,1.0,22.0,14.990000,0.750000,0.738986,True,train
5506591,5404,50013,0.487016,65.168657,0.711940,40271.402985,20.949254,28.095522,11.605970,9.023881,...,226,37.50,34.533148,1.0,75.0,18.750000,0.750000,1.085913,True,train
1479941,1402,56215,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,006,19.27,16.329684,1.0,49.0,34.028001,0.600000,1.180060,True,train
3844504,3607,52901,0.509057,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,...,103,2.10,4.035000,1.0,6.0,5.150000,0.750000,0.520446,True,train


In [66]:
test.head()

,STORE,COUNT_ITEMS,MenRatio,White,Asian,Income,Poverty,Professional,Construction,Unemployment,...,CLASSID,COST,RETAIL,avg_quan,avg_orig,avg_sprice,purchase_prob,PROF_MARG,DISCOUNTED,train_type
4630870,4403,67710,0.500793,89.511828,0.513978,51928.354839,11.195699,35.009677,14.298925,2.847312,...,301,10.635802,9.598889,1.0,34.0,14.8750,0.75,1.108024,True,test
163926,203,72625,0.498636,90.566957,0.631304,44435.304348,17.261739,29.287826,12.403478,6.026087,...,005,17.630000,38.643855,1.0,39.0,39.0000,0.80,0.456217,False,test
8113102,9103,101533,0.496768,91.810000,0.513333,41470.491667,21.304167,28.687500,12.188333,7.672500,...,108,13.500000,6.685000,1.0,29.0,24.6500,0.80,2.019447,True,test
3012004,2907,71103,0.509057,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,...,005,19.000000,14.865876,1.0,42.0,36.4975,0.75,1.278095,True,test
1109925,1004,59770,0.495863,86.431579,0.683158,43016.610526,18.455789,28.375789,11.535789,7.344211,...,111,22.887182,37.404545,1.0,78.0,40.9500,0.75,0.611882,True,test


In [67]:
df_comb = pd.concat([train,test])
df_dummies = pd.get_dummies(df_comb, columns=categorical_variables, drop_first=True)
df_dummies.head()

,STORE,COUNT_ITEMS,MenRatio,White,Asian,Income,Poverty,Professional,Construction,Unemployment,...,CLASSID_H0G,CLASSID_H0H,CLASSID_H0K,CLASSID_H1B,CLASSID_H1E,CLASSID_H4B,CLASSID_H4E,CLASSID_I0D,CLASSID_I9D,DISCOUNTED_True
8438940,9409,66634,0.517922,74.657813,1.109375,54353.562500,13.234375,35.031250,14.535937,5.793750,...,False,False,False,False,False,False,False,False,False,False
3248046,3107,44581,0.509057,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,...,False,False,False,False,False,False,False,False,False,True
5506591,5404,50013,0.487016,65.168657,0.711940,40271.402985,20.949254,28.095522,11.605970,9.023881,...,False,False,False,False,False,False,False,False,False,True
1479941,1402,56215,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,False,False,False,False,False,False,False,False,False,True
3844504,3607,52901,0.509057,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,...,False,False,False,False,False,False,False,False,False,True


In [68]:
df_train = df_dummies[df_dummies.train_type == 'train']
df_test = df_dummies[df_dummies.train_type == 'test']

print(df_train.shape, df_test.shape)

(1448232, 1291) (965490, 1291)


In [22]:
# TO DO 
"""
import torch
from transformers import BertModel, BertTokenizer

def get_brand_embedding(brand_name, model, tokenizer):
    # Tokenize the brand name
    inputs = tokenizer(brand_name, return_tensors="pt", truncation=True, padding="max_length", max_length=32)
    
    # Get the model's output. Output is a tuple, where the first item contains the embeddings
    with torch.no_grad():
        output = model(**inputs)
    
    # For BERT, the embeddings of the [CLS] token (first token) is often used as the sequence representation
    # Alternatively, you can use mean pooling or max pooling across all token embeddings
    embedding = output[0][:, 0, :]  # [CLS] token embedding
    
    return embedding.squeeze().numpy()

# Load the BERT model and tokenizer
model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# List of fashion brand names
brands = ["Gucci", "Prada", "Louis Vuitton"]

# Get embeddings for each brand
brand_embeddings = {brand: get_brand_embedding(brand, model, tokenizer) for brand in brands}

for brand, embedding in brand_embeddings.items():
    print(f"Embedding for {brand}:", embedding)

"""

'\nimport torch\nfrom transformers import BertModel, BertTokenizer\n\ndef get_brand_embedding(brand_name, model, tokenizer):\n    # Tokenize the brand name\n    inputs = tokenizer(brand_name, return_tensors="pt", truncation=True, padding="max_length", max_length=32)\n    \n    # Get the model\'s output. Output is a tuple, where the first item contains the embeddings\n    with torch.no_grad():\n        output = model(**inputs)\n    \n    # For BERT, the embeddings of the [CLS] token (first token) is often used as the sequence representation\n    # Alternatively, you can use mean pooling or max pooling across all token embeddings\n    embedding = output[0][:, 0, :]  # [CLS] token embedding\n    \n    return embedding.squeeze().numpy()\n\n# Load the BERT model and tokenizer\nmodel_name = "bert-base-uncased"\nmodel = BertModel.from_pretrained(model_name)\ntokenizer = BertTokenizer.from_pretrained(model_name)\n\n# List of fashion brand names\nbrands = ["Gucci", "Prada", "Louis Vuitton"]\n\n

In [36]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
import numpy as np

In [37]:
# Your models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor()
}


In [38]:
# Function to calculate adjusted R-squared
def adjusted_r2(r2, n, k):
    return 1 - ((1 - r2) * (n - 1)) / (n - k - 1)


In [39]:
df_train.head()

,STORE,COUNT_ITEMS,MenRatio,White,Asian,Income,Poverty,Professional,Construction,Unemployment,...,CLASSID_H0G,CLASSID_H0H,CLASSID_H0K,CLASSID_H1B,CLASSID_H1E,CLASSID_H4B,CLASSID_H4E,CLASSID_I0D,CLASSID_I9D,DISCOUNTED_True
4241807,4103,68311,0.500793,89.511828,0.513978,51928.354839,11.195699,35.009677,14.298925,2.847312,...,False,False,False,False,False,False,False,False,False,True
660532,509,72821,0.504501,50.826667,1.293333,45816.666667,20.193333,30.006667,13.153333,9.766667,...,False,False,False,False,False,False,False,False,False,True
6420725,7004,76352,0.493487,71.684211,2.062406,56270.037594,14.367669,34.354135,10.771429,6.033835,...,False,False,False,False,False,False,False,False,False,True
241417,302,76063,0.514077,68.120896,1.543284,47144.328358,17.150746,31.208955,12.074627,7.765672,...,False,False,False,False,False,False,False,False,False,False
6730863,7307,60898,0.509057,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,...,False,False,False,False,False,False,False,False,False,True


In [44]:
train['combined'] = train[categorical_variables].astype(str).agg('-'.join, axis=1)

In [45]:
X_orig = train.drop(['purchase_prob', 'train_type', 'combined'], axis=1).values
y_orig = train['purchase_prob'].values

# Set up the stratified K-fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [46]:
X_dum = df_train.drop(['purchase_prob', 'train_type'], axis=1).values
y_dum = df_train['purchase_prob'].values

In [47]:
# # Insert old names
# original_categorical_columns = ['Category1', 'Category2']

# # Dummy function to reconstruct the 'combined' column from the dummy-encoded data
# # In reality, you would need to have this mapping available from your encoding process
# def reconstruct_combined_feature(row, original_categorical_columns):
#     combined = []
#     for col in original_categorical_columns:
#         for dummy in row.index[row.astype(bool)]:
#             if dummy.startswith(col):
#                 category = dummy.split('_')[1]
#                 combined.append(category)
#     return '-'.join(combined)

# # Replace df with train
# df['combined'] = df.apply(lambda row: reconstruct_combined_feature(row, original_categorical_columns), axis=1)

# Define the features and target


# Perform stratified cross-validation
for name, model in models.items():
    mse_scores = []
    r2_scores = []
    adj_r2_scores = []

    for train_index, test_index in skf.split(X_orig, train['combined']):
        # Clone the model to ensure a fresh model for each fold
        cloned_model = clone(model)

        # Create the training and test sets
        X_train, X_test = X_dum[train_index], X_dum[test_index]
        y_train, y_test = y_dum[train_index], y_dum[test_index]

        # Fit the model
        cloned_model.fit(X_train, y_train)
        
        # Make predictions
        predictions = cloned_model.predict(X_test)
        
        # Calculate metrics
        mse = mean_squared_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        n = len(y_test)
        k = X_test.shape[1]
        adj_r2 = adjusted_r2(r2, n, k)
        
        mse_scores.append(mse)
        r2_scores.append(r2)
        adj_r2_scores.append(adj_r2)
    
    # Print the average scores across the folds
    print(f"{name}:")
    print(f"Mean MSE: {np.mean(mse_scores):.3f}")
    print(f"Mean R-squared: {np.mean(r2_scores):.3f}")
    print(f"Mean Adjusted R-squared: {np.mean(adj_r2_scores):.3f}\n")

# Clean up the 'combined' column 
#df = df.drop('combined', axis=1)


/Users/dare_devil/news_env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: 